<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/old/BP_judge_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화 중 질문을 선별하는 코드(로컬)

In [ ]:
!pip install soynlp
!pip install datasets
!pip install accelerate -U
!pip install kss
!pip install evaluate
import evaluate
import accelerate
import random
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
import torch.nn as nn
from kss import Kss
from transformers.modeling_outputs import SequenceClassifierOutput
from datasets import Dataset, load_dataset, ClassLabel, load_metric
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from tqdm import tqdm
from torch.nn.functional import cross_entropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoModel, AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM, Trainer, TrainerCallback, BertConfig, BertForSequenceClassification, RobertaConfig
from sklearn.model_selection import StratifiedKFold, train_test_split
from evaluate import evaluator
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [ ]:
print(torch.version.cuda)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

12.1


In [ ]:
# 모델과 토크나이저 초기화
HUGGINGFACE_MODEL_PATH = "bespin-global/klue-roberta-small-3i4k-intent-classification"
tokenizer = RobertaTokenizerFast.from_pretrained(HUGGINGFACE_MODEL_PATH)
pretrained_model = RobertaForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/272M [00:00<?, ?B/s]

In [ ]:
#출력 레이블 개수를 바꾸기 위해 출력층 하나 더 쌓기
class ExtendedRobertaForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model):
        super(ExtendedRobertaForSequenceClassification, self).__init__()
        # 기존 파인튜닝 모델 로드
        self.pretrained_model = pretrained_model
        self.config = RobertaConfig(
            _name_or_path = "BP_judge_question",
            architectures = ["RobertaForSequenceClassification"],
            attention_probs_dropout_prob = 0.1,
            bos_token_id = 0,
            eos_token_id = 2,
            hidden_act = "gelu",
            hidden_dropout_prob = 0.1,
            hidden_size = 768,
            id2label = {0: "not-question", 1: "question"},
            initializer_range = 0.02,
            intermediate_size = 3072,
            label2id = {"question": 1, "not-question": 0},
            layer_norm_eps = 1e-05,
            max_position_embeddings = 514,
            model_type = "roberta",
            num_attention_heads = 12,
            num_hidden_layers = 6,
            pad_token_id = 1,
            position_embedding_type = "absolute",
            tokenizer_class = "BertTokenizer",
            type_vocab_size = 1,
            use_cache = True,
            vocab_size = 32000,
            num_labels = 2
        )  # 기존 모델의 config를 수정 후 저장

        #새로운 출력층 직전의 비선형 활성화 함수로 GELU 사용
        self.additional_activation = nn.GELU()

        # 새 태스크를 위한 추가 출력층, 7개 출력을 받아서 2개로 분류
        self.additional_classifier = nn.Linear(7, 2)

    #AttributeError: 'ExtendedRobertaForSequenceClassification' object has no attribute 'can_generate' 해결용 함수. 이유 아직 모름
    def can_generate(self):
        return False  # 메서드로 변경

    def forward(self, input_ids, attention_mask=None, labels = None):
        # 기존 모델을 통해 입력 데이터 처리
        outputs = self.pretrained_model(input_ids, attention_mask=attention_mask)

        # #GELU 층에 통과
        # probs = self.additional_activation(outputs.logits)

        # # 새로운 태스크의 예측을 위한 추가 출력층 사용
        # logits = self.additional_classifier(probs)

        #직전 비선형 활성화 함수 없을 때 코드
        logits = self.additional_classifier(outputs.logits)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

# 확장된 모델 인스턴스 생성
model = ExtendedRobertaForSequenceClassification(pretrained_model)

In [ ]:
#GPU 사용 코드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # GPU 메모리 사용량 확인 (필요시)
# print(torch.cuda.memory_summary(device=device, abbreviated=True))

#파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
model.to(device)

ExtendedRobertaForSequenceClassification(
  (pretrained_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaS

In [ ]:
#전처리된 세 파일을 데이터프레임으로 로드
df_qna = pd.read_excel('/content/drive/My Drive/community_qna_preprocessed.xlsx')
df_all_contents = pd.read_excel('/content/drive/My Drive/community_all_contents_preprocessed.xlsx')
df_all_comments = pd.read_excel('/content/drive/My Drive/community_all_comments_preprocessed.xlsx')

왜 빌더로그 게시글 요소들을 세 부분으로 나눠야 하는가?
1. positive sample, negative sample 비율 1대1이 되도록 만들기
2. 사용할 RobertaForSequenceClassification 아키텍처를 보면 (position_embeddings): Embedding(514, 768, padding_idx=1)이 포함되어 있다. negative sample에 해당하는 빌더로그 게시글만 길이가 길다면 이러한 정보까지 학습할 여지가 있다.

왜 tokenizer()로 바로 시퀀스 길이를 구하지 않고 원본 리스트부터 시작해서 평균 길이를 구하는가
->

인자 중 max_length가 존재하기 때문에 어차피 길이를 초과하는 데이터는 truncation이 진행된다.

그렇기 때문에 데이터 평균 길이를 비교하려면 원본 리스트를 사용해야 한다.

In [ ]:
split_sentences = Kss('split_sentences')

In [ ]:
#데이터셋의 tokenized data의 길이를 구하는 함수
def tokenized_sequence_length(text):
  #kss 모듈로 글을 문장으로 분리
  original_sentences = split_sentences(text)
  index_sentences = tokenizer(original_sentences)['input_ids']
  index_sentences_filtered = [sentence[1:-1] for sentence in index_sentences] #cls_token과 sep_token 제외
  total_length = sum(len(s) for s in index_sentences_filtered)
  return index_sentences_filtered, total_length

In [ ]:
#데이터셋에서 각 요소를 n등분하는 함수
def divide_text_to_number(text, number):
  #kss 모듈로 문장으로 분리
  index_sentences, total_length = tokenized_sequence_length(text)
  target_length = total_length / number

  # 세 부분으로 분할
  segments = []
  current_segment = []
  current_length = 0

  #마지막 세그먼트 추가를 위해 사용하는 인덱스 값
  sentence_checkpoint = 0

  for index, sentence in enumerate(index_sentences):
      if current_length + len(sentence) > target_length and current_segment:
          segments.append(sum(current_segment, []))
          current_segment = [sentence]
          current_length = len(sentence)
          if len(segments) == 2:  # 이미 두 세그먼트를 만들었다면 나머지는 모두 마지막 세그먼트에 추가
              sentence_checkpoint = index #current_segment에 추가해둔 문장을 인덱싱
              break
      else:
          current_segment.append(sentence)
          current_length += len(sentence)

  # 마지막 세그먼트 추가
  segments.append(sum(current_segment + index_sentences[sentence_checkpoint + 1:], []))

  return tokenizer.batch_decode(segments)


In [ ]:
#BP_judge_question용 데이터 제작

#slug가 있다면 질문, 없다면 답변, 이 중 질문을 positive sample로 설정
question_list = df_qna[pd.isna(df_qna['Slug']) == False]['내용'].dropna(axis = 0).to_list()
label_question = [1] * len(question_list)

print(question_list)
print(len(question_list))

#질문 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(question) for question in question_list) / len(question_list))

#질문 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in question_list) / len(question_list))

['안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 쓰게 되네요!서베이 폼을 이용해 받은 설문 내용을 노션 데이터베이스로 가져온 후 자피어를 활용하여 데이터베이스의 각 속성 값을 새로운 페이지의 글로 작성하려 하는데 일반적인 딱딱한 텍스트만 있는 글이 아니라 기본적으로 노션에서 사용할 수 있는 콜아웃 블록이나 인용 등의 기능을 구현하고 싶은데 자피어에서 새로운 페이지를 작성하는 필드에 노션 기능을 구현할 수 있는 명령어나 언어가 따로 있는지 궁금해서 이렇게 질문을 남깁니다.이 고민을 몇시간째 하고 있는데 사이트 오픈 소식 메일이 와서 사이트를 둘러 보다가 이렇게 글 남깁니다.다시 한번 오픈 축하드립니다.감사합니다.', '안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 페이지를 웹뷰로보면 양측면으로 여백이 있는데요, 제가 현재 개발하고 있는 페이지에는 여백이 적용되지않습니다.각 페이지마다 사이드그룹을 지정해서 여백을 두신건지.. 다른 세팅 방법이 있는지 궁금합니다.현재 제가 작업중인 페이지를 이미지로 첨부합니다. 헤더가 왼쪽에서 오른쪽까지 가득 차있어요. (혹시 테스트뷰만 이렇게 나오는건 아닐까라는 기대도 해보게되네요..)', '안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나 이후로 자극 받아 버블로 다시 시작하려 합니다!사실 노션에서 기능이 너무 한정적이라 답답함이 있었고, 특정 기능은 굉장히 높은 코딩 스킬을 요구한다는 것을 알게 되었습니다ㅜMVP앱 기능을 아래처럼 구상중입니다.사용자 자기소개서(서베이 폼 Tally) 작성사용자 응답에서 각 요소별 답변을 데이터베이스에 저장데이터베이스를 제가공(자동화?)하여 프로필 페이지 생성프로필 내용 일부 공개, 전체 열람 유료 결제1:1 대화신청 (카카오톡 1:1 대화방 or 자체 채팅기능?) 유료 결제답변 요소 중 특정 요소만 따로 모아 놓은 페이지 생성 (예, 유튜브 추천 영상 모음 페이지)매주 1회 답변 요소 중

Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors


토큰화된 텍스트 평균 길이 : 138.38368580060424


In [ ]:
#답변을 negative sample로 설정
answer_list = df_qna['답변'].dropna(axis = 0).to_list()
label_answer = [0] * len(answer_list)

print(answer_list)
print(len(answer_list))

#답변 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(answer) for answer in answer_list) / len(answer_list))

#답변 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in answer_list) / len(answer_list))

['저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못했고,마크다운을 지원하기 때문에 토글이나 체크리스트, 인용구 같은 부분까지는 꾸밀 수 있는 것 같습니다. 블로그에 자세히 나와있는 것 같아요.콘텐츠 부분에 마크다운을 미리 채워두시고 내용물은 동적 데이터로 지정하시면 깔끔하게 떨어지지 않을까 싶네요. (혹시 이미 해보셨으려나요?;;)저도 해보진 않은 거라서 어디까지나 추측이고 가능성입니다. 정민님이 해보시고 도움 되셨다면 빌더 로그 같은 곳에 공유 부탁드립니다!', '헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요! 그리고 헤더그룹의 Horizontal Alignment를 centered로 두시면 그룹이 페이지 중앙에 위치하고 설정해두신 사이즈까지만 늘어나면서 여백이 생길 것 같아요!화면 해상도 관련해서는 아래에서 참고하시면 될 것 같습니다 , 모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 경우 페이지 width를 1080px로 하고 바디 그룹의 레이아웃을 row로 했고, 다시 그 안에 column 그룹을 2개 추가하여 레이아웃을 다음 이미지와 같이 짰습니다~', '충분히 버블로 개발가능할 것 같아요! 이미 비슷한 수준의 어플을 개발한 사례가 많아서 참고하실 레퍼런스도 많을 것 같아요.혹시 시간이 걸리실 것 같으면 매칭, 채팅 같은 핵심기능만 먼저 개발하셔서 테스트 돌려보시면서 지표 보시는 것도 좋을 것 같아요! 마침 tinder를 클론할 수 있도록 버블이 가이드를 제공한 페이지가 있어 공유드립니다. MVP 출시하시면 꼭 알려주세요 ㅎㅎ 제주도 사는 친구들에게 공유해보고 싶네요 :)', '폴리미 가입해야지 가입해야지 하다가 바로 가입했습니다. 정신이 없었네요. 좋은 프로젝트라서 기대가 됩니다. 따지고보면 저도 모두의노코드 모더레이터가 부캐거든요!1) 모두의노코드는 페이지별로 모바일용 페이지를 따로 만드셨나요?A : 중요한 페이지만 모바일용으로 따로 만들었습

In [ ]:
#빌더로그 글을 negative sample로 설정
builder_log_list = df_all_contents[df_all_contents['커뮤니티 타입'] == '빌더 로그']['내용'].dropna(axis = 0).to_list()

print(builder_log_list)
print(len(builder_log_list))

#빌더로그 글 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(builder_log) for builder_log in builder_log_list) / len(builder_log_list))

#빌더로그 글 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list) / len(builder_log_list))

#시퀀스 길이 균형을 위해 각 빌더로그 글을 3등분
builder_log_list_divided = sum([divide_text_to_number(text, 3) for text in builder_log_list], [])

#분할한 데이터의 개수에 맞춰서 레이블 설정
label_builder_log = [0] * (len(builder_log_list_divided))

#분리된 빌더로그 글 평균 길이 확인(토큰화 결과)
print('분리된 데이터셋에서 토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list_divided) / len(builder_log_list_divided))

["어제 프론티어 분들께만 모두의노코드 오픈 소식을 하루 먼저 알려드리고 불과 몇 시간 만에 서버 용량이 거의 찼다는 메일을 받았습니다.웨비나 중이어서 메일을 미처 확인하지 못했는데요. 약 30여 명 정도 가입한 시점에 서버 용량이 다 차버릴 줄은 꿈에도 몰랐습니다. 퍼스널 요금제가 정말 그야말로 '퍼스널'이었네요.그래서 5분 정도가 가입하실 때 워크플로우가 제대로 작동하지 않아 가입은 되셨는데 데이터가 제대로 들어가지 않았습니다. 심심한 사과의 말씀드립니다. 이분들은 따로 연락을 드려 조치를 취할 예정입니다.정리 : 커뮤니티 제작하실 때 되도록 퍼스널 요금제는 피하시는 게 좋을 것 같습니다.", '슬랙 알림 기능은 비교적 간단하며, 무료입니다.api connector을 이용하며, 이것을 이용해 워크플로우 상에서 특정 채널에 내가 원하는 메세지를 보낼 수 있습니다.먼저 Plugin에서 API Connector을 설치하세요위와 같은 형태로 api를 추가하면 됩니다. 빨간색 박스로 표시한 설정을 아래와 같이 따라해보세요.Use as ‘Action’Data type ‘Text’POST (드롭다운)Body type : JSONBody : {”text”: “<text>” }Body parameters - value : 아무 텍스트 (테스트용이라서 아무렇게나 입력하셔도 됩니다), Private 체크 해제이제 초록색 부분에 api url을 입력하면 되는데, 핑크색 가려진 부분은 내가 슬랙 메세지를 보내고 싶은 채널의 고유 id를 가져와야 합니다. 방법은 아래에서 계속.위 링크로 접속하면, 수신 웹후크 앱이 나옵니다. 이제 이 앱을 ‘Slack에 추가’ 합니다.로그인 한 상태에서, 포스트할 채널을 드롭다운에서 선택합니다.이제 웹후크 URL이라는 것이 나옵니다. 이것을 앞에서 설명드린 초록색 박스에 복사붙여넣기 하면 됩니다.Initialize를 누르면, ok 라는 텍스트가 나옵니다. (정상적으로 완료된 상태)이제 워크플로우 상에서 등록한 api를 사용해 슬랙에 메세지를 보낼 

In [ ]:
#데이터프레임 생성
text = question_list + answer_list + builder_log_list_divided
label = label_question + label_answer + label_builder_log

df_judge_question = pd.DataFrame([text, label], index = ['text', 'label'])
df_judge_question = df_judge_question.T
print(df_judge_question)

                                                  text label
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...     1
1    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...     1
2    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...     1
3                                 API키는 분리해서 쓰시면 좋겠습니다     1
4                                                    .     1
..                                                 ...   ...
676  수많은 에이전시들을 보다보면 어떤 기준으로 에이전시를 골라야 하는지 감을 잡기가 어...     0
677  바로 시간당 비용을 보는 것인데요 에이전시 프로필 페이지에 가보시면 이렇게 시간당 ...     0
678  스티비는 국내 뉴스레터 서비스로, 카페24, 고도몰, 식스샵 등 다양한 서비스와 연...     0
679   하지만 버블에서 딱히 연동할 수 있는 게 없어서 무료 플러그인을 작업하고 있습니다. _     0
680  x0008 _ 이번 주에 무료 배포할 예정입니다 : ) + 필요한 플러그인이 있다면...     0

[681 rows x 2 columns]


In [ ]:
print(len(df_judge_question[df_judge_question['label'] == 0]))

350


In [ ]:
#과적합 방지를 위한 EarlyStopping 클래스
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 검증 손실 가져오기
        current_loss = kwargs['metrics']['eval_loss']

        # 최고 성능 갱신 또는 카운터 증가
        if self.best_score is None:
            self.best_score = current_loss
        elif current_loss > self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
        else:
            self.best_score = current_loss
            self.counter = 0

In [ ]:
#데이터셋 전처리 함수
# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True, padding = "max_length", max_length = model.config.max_position_embeddings)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding = 'max_length', max_length = 512)

# 공통 테스트 데이터셋 준비

In [ ]:
#훈련 데이터 불러오기
finetune_dataset_path = '/content/drive/My Drive/KcBERT_finetune_dataset_question.xlsx'
df_finetune = pd.read_excel(finetune_dataset_path, index_col = 0)

In [ ]:
#훈련 데이터셋의 결측치 제거
print(df_finetune[pd.isna(df_finetune['text']) == True])
df_finetune = df_finetune[pd.isna(df_finetune['text']) == False]
print(df_finetune)

    text  label
814  NaN      0
815  NaN      0
816  NaN      0
817  NaN      0
818  NaN      0
..   ...    ...
991  NaN      0
992  NaN      0
993  NaN      0
994  NaN      0
995  NaN      0

[182 rows x 2 columns]
                                                   text  label
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...      1
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...      1
2     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...      1
3                                  API키는 분리해서 쓰시면 좋겠습니다      1
4                                                     .      1
...                                                 ...    ...
3584   안녕하세요. 왕초보자입니다. 리피팅 그룹을 따라 만들어보고 있는데 저는 스크롤에 ...      1
3585                                 영상의 에디터가 옛날버전인것같네요      0
3586   그래서 저는 이것저것에서 스크롤 기능을찾아보는데 없네요... 업데이트 되면서 리피...      1
3587   지금은 1*4로. 리피팅 그룹이 고정된 상태시네요 가로 스크롤을 구현하려고 하시는...      0
3588             그럼 row값을 1로 고정하시고 column은 Fixed를 푸셔야해요      0

[3407 rows x 2 columns]


In [ ]:
#파인튜닝을 위한 데이터셋 준비, 훈련/검증 8:2 분리
third_dataset = Dataset.from_pandas(df_finetune)
third_finetune_dataset = third_dataset.map(preprocess_function, batched=True)
third_finetune_dataset = third_finetune_dataset.class_encode_column('label')
third_finetune_dataset = third_finetune_dataset.train_test_split(test_size = 0.2, stratify_by_column = 'label')

Map:   0%|          | 0/3407 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/3407 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/3407 [00:00<?, ? examples/s]

In [ ]:
#성능 측정을 위한 데이터 추가 분리

#학습에 사용할 훈련 데이터, (검증 + 테스트) 데이터 분리
third_train_dataset, third_valid_test_dataset = third_finetune_dataset['train'], third_finetune_dataset['test']

#(검증 + 테스트) 데이터를 1:1로 분리
#이때 분리된 350개의 데이터를 공통 테스트 데이터로 활용함.
third_valid_test_dataset = third_valid_test_dataset.train_test_split(test_size = 0.5, stratify_by_column = 'label')
third_valid_dataset, question_test_dataset = third_valid_test_dataset['train'], third_valid_test_dataset['test']

# 데이터셋:540 + 60

In [ ]:
#파인튜닝을 위한 데이터셋 준비, 훈련/검증 9:1 분리
first_dataset = Dataset.from_pandas(df_judge_question)
first_finetune_dataset = first_dataset.map(preprocess_function, batched=True)
first_finetune_dataset = first_finetune_dataset.class_encode_column('label')
first_finetune_dataset = first_finetune_dataset.train_test_split(test_size = 0.1, stratify_by_column = 'label')

Map:   0%|          | 0/681 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/681 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/681 [00:00<?, ? examples/s]

In [ ]:
print(set(first_finetune_dataset['train']['label']))

{0, 1}


In [ ]:
#훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    learning_rate = 5e-5,
    evaluation_strategy = 'steps',
    eval_steps = 50,
    save_strategy = "steps",
    save_steps = 50,
    num_train_epochs = 3,
    save_total_limit = 3,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = first_finetune_dataset['train'],
    eval_dataset = first_finetune_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 3)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.540375
100,No log,0.228149
150,No log,0.218051
200,No log,0.104842


TrainOutput(global_step=231, training_loss=0.2283858922652868, metrics={'train_runtime': 113.5101, 'train_samples_per_second': 16.175, 'train_steps_per_second': 2.035, 'total_flos': 0.0, 'train_loss': 0.2283858922652868, 'epoch': 3.0})

# 데이터셋:540 + 60 + kfold

In [ ]:
# #(훈련+검증)/테스트 9:1 분리
# X, y = df_judge_question["text"], df_judge_question["label"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle = True, stratify = y) #초기에는 정렬된 데이터셋으로 stratify로 레이블 비율을 유지하며 데이터셋을 섞은 후 분리

# #pandas dataframe으로 변환
# train_df = pd.DataFrame({"text": X_train, "label": y_train})
# test_df = pd.DataFrame({"text": X_test, "label": y_test})

# # Hugging Face의 Dataset 객체로 변환
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

#pandas dataframe으로 변환
train_df = pd.DataFrame({"text": df_judge_question['text'], "label": df_judge_question['label']})

# Hugging Face의 Dataset 객체로 변환
second_dataset = Dataset.from_pandas(train_df)

second_finetune_dataset = second_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/681 [00:00<?, ? examples/s]

In [ ]:
#분류 문제이며 레이블 간 데이터 수 비율을 고려하기 위해 StratifiedKFold 사용

# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(skf.split(second_finetune_dataset, second_finetune_dataset['label'])):

    # 훈련 세트와 검증 세트 분리
    second_train_dataset = second_finetune_dataset.select(train_idx)
    second_val_dataset = second_finetune_dataset.select(val_idx)

    # 훈련 설정
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy = "steps",
        eval_steps = 50,
        save_strategy = "steps",
        save_steps = 50,
        learning_rate = 5e-5,
        num_train_epochs = 3, #고려사항
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        weight_decay = 0.01, #고려사항
        logging_dir = "./logs",
        load_best_model_at_end = True

    )

    # 트레이너 초기화 및 훈련
    #RobertaForSequenceClassification 모델은 기본적으로 분류 문제에 적합한 loss function을 내장하고 있습니다. num_labels=2로 설정함으로써, 이 모델은 이진 분류 문제를 위해 사전 구성되며, 내부적으로 Cross-Entropy Loss를 사용합니다.
    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = second_train_dataset,
        eval_dataset = second_val_dataset,
        callbacks = [EarlyStoppingCallback(patience = 3)]
    )

    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.390818
100,No log,0.408227
150,No log,0.386588
200,No log,0.444056


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.180735
100,No log,0.117956
150,No log,0.129505
200,No log,0.143581


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.027822
100,No log,0.070295
150,No log,0.086129
200,No log,0.080546


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.127445
100,No log,0.129566
150,No log,0.093335
200,No log,0.095400


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.000672
100,No log,0.037016
150,No log,0.001277
200,No log,0.001192


# 데이터셋:3150 + 350

In [ ]:
#훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    learning_rate = 5e-5,
    evaluation_strategy = 'steps',
    eval_steps = 100,
    save_strategy = "steps",
    save_steps = 100,
    num_train_epochs = 3,
    save_total_limit = 3,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 100, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = third_train_dataset,
    eval_dataset = third_valid_dataset,
    callbacks = [EarlyStoppingCallback(patience = 3)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
100,No log,0.277275
200,No log,0.309800
300,No log,0.302082
400,No log,0.372051


TrainOutput(global_step=400, training_loss=0.3028225898742676, metrics={'train_runtime': 244.0604, 'train_samples_per_second': 33.496, 'train_steps_per_second': 4.192, 'total_flos': 0.0, 'train_loss': 0.3028225898742676, 'epoch': 1.1730205278592376})

# 성능 테스트

In [ ]:
# Metric 로드
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# compute_metrics 함수 정의
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }

In [ ]:
#테스트 데이터셋 준비
tokenized_test_dataset = question_test_dataset.map(preprocess_function, batched=True)

#테스트 데이터셋으로 모델 평가
trainer = Trainer(
    model = model,
    compute_metrics = compute_metrics,
)

trainer.predict(tokenized_test_dataset)

Map:   0%|          | 0/341 [00:00<?, ? examples/s]

PredictionOutput(predictions=array([[ 2.5722337 , -1.2674041 ],
       [ 2.4155598 , -1.3023831 ],
       [ 2.1778972 , -1.5896902 ],
       [-2.0503058 ,  0.07196289],
       [ 2.5216875 , -1.3985614 ],
       [ 2.5732431 , -1.2177005 ],
       [ 2.5213966 , -1.1294053 ],
       [ 2.1912982 , -1.4992548 ],
       [ 2.385707  , -1.3431473 ],
       [ 2.3534966 , -1.403205  ],
       [-0.38186902, -2.5823812 ],
       [ 2.3390727 , -1.4441649 ],
       [-1.9839998 ,  0.9401125 ],
       [ 2.419053  , -1.4541211 ],
       [ 2.597292  , -1.1593791 ],
       [ 2.6777635 , -1.2606609 ],
       [-2.0087304 ,  1.3821634 ],
       [ 2.0223384 , -1.6912296 ],
       [ 0.72150147, -1.7551312 ],
       [ 0.3791026 ,  2.1585965 ],
       [ 2.744615  , -1.1219299 ],
       [-0.5505212 , -0.3918211 ],
       [ 2.1697614 , -1.5800681 ],
       [ 2.6329174 , -1.290102  ],
       [ 2.107398  , -1.5641874 ],
       [ 2.7374086 , -0.9798056 ],
       [-2.2892914 ,  0.24417897],
       [ 2.1461692 , -1.57

# 실제 데이터에 적용

In [ ]:
#카카오톡 대화내용 불러오고 데이터프레임으로 받기
file_path = '/content/drive/My Drive/talk_preprocess_result_long.csv'

df = pd.read_csv(file_path)

In [ ]:
#데이터 처리 Pipeline 설정, top_k를 1로 설정시 최대로 측정된 label만 출력됨.
#텍스트 처리해보니 시퀀스 길이가 539인 행 존재 -> truncation 진행, 최대 길이는 512
text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    top_k = 1,
    truncation = True,
    batch_size = 128,
    device = device
)

In [ ]:
# #결측치 제거
# df['text'] = df['text'].dropna(axis = 0)

In [ ]:
# #결측치 검증
# print(type(text[:5]))
# print(all(isinstance(item, str) for item in text))
# for i, item in enumerate(text):
#   if not isinstance(item, str):
#     print(f"Index {i}: {item} (Type: {type(item)})")

In [ ]:
#결측치 제거
df = df.replace('None', np.nan).dropna(axis = 0)

#모델로 prediction_labels 리스트 생성.
text = df['text'].to_list()

#tqdm으로 진행률 표시
predictions= []
for i in tqdm(range(0, len(text), 128), desc='Classifying'):
    batch = text[i:i+128]
    predictions.extend(text_classifier(batch))

# predictions = text_classifier(text)
prediction_labels = [prediction[0]['label'] for prediction in predictions]

#Dataframe에 새 열로 추가.
df['label'] = prediction_labels

Classifying: 100%|██████████| 135/135 [01:54<00:00,  1.18it/s]


In [ ]:
#.csv 파일로 google drive에 저장
#illegalcharactererror에 의해 .xlsx 대신 .csv로 저장
save_path = '/content/drive/My Drive/judge_question_result_long.csv'

df.to_csv(save_path)